In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [2]:
print dr.train
print dr.artists

[[ 8890 22426 54743 ..., 32763 32763 32763]
 [71058 19741 71298 ..., 87949 62773 70493]]
[[77187 69189 37833 ..., 64702 57834 68922]
 [    2     4     2 ...,   980 14291 17533]]


In [98]:
dic_ar = {}
ar_tr = dr.artists.T
for row in ar_tr:
    dic_ar[row[0]] = row[1]

n_playlists = dr.n_playlists
n_attributes = dr.n_attributes

print n_attributes
print n_playlists

train = dr.train.T
r,c,v = [],[],[]
for row in train:
    r.append(row[0])
    c.append(dic_ar[row[1]])
    v.append(1)

icm_ar = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

77040
57561


In [3]:
#playlist-artist
model = BM25Recommender(K=55)
model.fit(icm_ar)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

NameError: name 'icm_ar' is not defined

In [97]:
dic_al = {}
al_tr = dr.albums.T
for row in al_tr:
    dic_al[row[0]] = row[1]
    
n_playlists = dr.n_playlists
n_attributes = dr.n_attributes

print n_attributes
print n_playlists

train = dr.train.T
r,c,v = [],[],[]
for row in train:
    if row[1] not in dic_al: continue
    r.append(row[0])
    c.append(dic_al[row[1]])
    v.append(1)

icm_al = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

77040
57561


In [109]:
#playtlist-album
model = BM25Recommender(K=100)
model.fit(icm_al)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

0.0704124722222


In [107]:
r,c,v = [],[],[]

for row in train:
    r.append(row[0])
    c.append(dic_ar[row[1]])
    v.append(1)

for row in train:
    if row[1] not in dic_al: continue
    r.append(row[0])
    c.append(dic_al[row[1]])
    v.append(1)

icm_al_ar = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

In [108]:
model = BM25Recommender(K=80)
model.fit(icm_al_ar)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

0.0716002777778


In [27]:
ucm1 = dr.getUCM_csr(artists=True, albums=False)
ucm1.data = np.ones(len(ucm1.data))
ucm2 = dr.getUCM_csr(artists=True, albums=False)
print ucm1.shape

(57561, 77040)


In [15]:
a = ucm1.sum(axis=1)
b = ucm2.sum(axis=1)

In [53]:
count = 0
p_target = []
for i in range(len(a)):
    if a[i]<=1 and b[i]>=5:
        if i in dr.target_playlists:
            p_target.append(i)
            count +=1
print count

374


In [37]:
ucm3 = dr.getUCM_csr(artists=False, albums=True)
ucm3.data = np.ones(len(ucm3.data))
ucm4 = dr.getUCM_csr(artists=False, albums=True)
print ucm3.shape

(57561, 77040)


In [38]:
c = ucm3.sum(axis=1)
d = ucm4.sum(axis=1)

In [43]:
count = 0
for i in range(len(c)):
    if c[i]<=1 and d[i]>=5:
        if i in dr.target_playlists:
            count +=1
print count

15
